# Fit model atmosphere to exoplanet spectrum

In [ ]:
#@title Imports
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os

## Get planet spectrum

In [ ]:
! wget https://share.phys.ethz.ch/~ipa/exoplanet_lecture_FS24/FIREFLy_transit_spec.h5

In [ ]:
# load data
with h5py.File("FIREFLy_transit_spec.h5", "r") as f:
    print("Keys: %s" % f.keys())
    wav = f['wavelength'][()]
    transit_depth = f['transit_depth'][()]
    transit_depth_uncertainty = f['transit_depth_uncertainty'][()]

In [ ]:
# visualise the spectrum


In [ ]:
#@title Install petitRADTRANS
!pip install meson-python ninja
!pip install petitRADTRANS --no-build-isolation

## Import petitRADTRANS package

In [ ]:
import petitRADTRANS
from petitRADTRANS.radtrans import Radtrans
from petitRADTRANS import physical_constants as cst

In [ ]:
from petitRADTRANS.config.configuration import petitradtrans_config_parser

In [ ]:
# Define input folder for opacities

## e.g creating a new folder 'input_data':
# os.mkdir("input_data")
# make sure all the opacity files are included and respective folder structures are kept!

## then set folder in petitRADTRANS config file
# petitradtrans_config_parser.set_input_data_path('./')

## if you are using your google drive to access the input_data folder
## first mount your drive
#from google.colab import drive
#drive.mount('/content/drive')

## set the folder path to
#petitradtrans_config_parser.set_input_data_path('./drive/MyDrive/')

## Initialize Radtrans Object
For this opacity files need to be loaded from the input_data folder.


In [ ]:
#Load opcities
radtrans = Radtrans(
pressures=np.logspace(-6, 2, 100),
line_species=[
    'H2O',
    'CO-NatAbund',
    'CH4',
    'CO2',
    'Na',
    'K'
],
rayleigh_species=['H2', 'He'],
gas_continuum_contributors=['H2-H2', 'H2-He'],
wavelength_boundaries=[0.3, 6]
)

### Get planet data

In [ ]:
from petitRADTRANS.planet import Planet

In [ ]:
planet = Planet.get('WASP-39 b')

In [ ]:
# Display the planet radius and its uncertainties
print(
    f"{planet.name}'s radius: {planet.radius * 1e-5:.0f} "
    f"+{planet.radius_error_upper * 1e-5:.0f} / {planet.radius_error_lower * 1e-5:.0f} km"
)

## Calculate first atmosphere

In [ ]:
temperatures = 1170 * np.ones_like(radtrans.pressures) # note that radtrans.pressures is in cgs units now, multiply by 1e-6 to get bars
mass_fractions = {
    'H2': 0.74 * np.ones(temperatures.size),
    'He': 0.24 * np.ones(temperatures.size),
    'H2O': 5e-5 * np.ones(temperatures.size),
    'CO-NatAbund': 1e-9 * np.ones(temperatures.size),
    'CO2': 1e-5 * np.ones(temperatures.size),
    'CH4': 1e-7 * np.ones(temperatures.size),
    'Na': 1e-4 * np.ones(temperatures.size),
    'K': 1e-6 * np.ones(temperatures.size)
}

#  2.33 is a typical value for H2-He dominated atmospheres
mean_molar_masses = 2.33 * np.ones(temperatures.size)

In [ ]:
wavelengths, transit_radii, _ = radtrans.calculate_transit_radii(
    temperatures=temperatures,
    mass_fractions=mass_fractions,
    mean_molar_masses=mean_molar_masses,
    reference_gravity=planet.reference_gravity,
    planet_radius=planet.radius,
    reference_pressure=planet.reference_pressure
)

## Visualise model atmosphere

In [ ]:
# plot observational data and model spectrum



## Test molecules one by one

In [ ]:
# eg H2O
radtrans = Radtrans(
    pressures=np.logspace(-6, 2, 100),
    line_species=[
        'H2O',
         ],
    rayleigh_species=['H2', 'He'],
    gas_continuum_contributors=['H2-H2', 'H2-He'],
    wavelength_boundaries=[0.3, 6]
)

In [ ]:
# plot observational data and model spectra
# plot spectra with varying H2O abundances

In [ ]:
# test the imapct of varying CO2 abudnances on the spectrum and visualize it

## More realistic PT profile

In [ ]:
from petitRADTRANS.physics import temperature_profile_function_guillot_global

In [ ]:
pressures_bar = radtrans.pressures * 1e-6 # cgs to bar
infrared_mean_opacity = 0.01
gamma = 0.4
intrinsic_temperature = 200
equilibrium_temperature = 1200

temperatures = temperature_profile_function_guillot_global(
    pressures=pressures_bar,
    infrared_mean_opacity=infrared_mean_opacity,
    gamma=gamma,
    gravities=planet.reference_gravity,
    intrinsic_temperature=intrinsic_temperature,
    equilibrium_temperature=equilibrium_temperature
)

In [ ]:
# plot PT profile

In [ ]:
mass_fractions = {
    'H2': 0.74 * np.ones(temperatures.size),
    'He': 0.24 * np.ones(temperatures.size),
    ## ?? complete with moelcules you want to include
}

#  2.33 is a typical value for H2-He dominated atmospheres
mean_molar_masses = 2.33 * np.ones(temperatures.size)

In [ ]:
power_law_opacity_350nm = 0.008
power_law_opacity_coefficient = -1.

wavelengths, transit_radii, _ = radtrans.calculate_transit_radii(
    temperatures=temperatures,
    mass_fractions=mass_fractions,
    mean_molar_masses=mean_molar_masses,
    reference_gravity=planet.reference_gravity,
    planet_radius=planet.radius,
    reference_pressure=planet.reference_pressure,
    power_law_opacity_350nm=power_law_opacity_350nm,
    power_law_opacity_coefficient=power_law_opacity_coefficient
)

In [ ]:
# plot observational data and model spectrum